In [1]:
import numpy as np
import cv2
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [2]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [3]:
cifar_data1 = unpickle("cifar-10-batches-py/data_batch_1")
cifar_data1.keys()

dict_keys([b'batch_label', b'labels', b'data', b'filenames'])

In [4]:
images = cifar_data1[b'data']
labels = cifar_data1[b'labels']

In [5]:
for data_batch in ["cifar-10-batches-py/data_batch_2",
                   "cifar-10-batches-py/data_batch_3",
                   "cifar-10-batches-py/data_batch_4",
                   "cifar-10-batches-py/data_batch_5"]:
    data_dict = unpickle(data_batch)

    images = np.append(images, data_dict[b'data'], axis=0)
    labels = labels + data_dict[b'labels']

In [6]:
test_data = unpickle("cifar-10-batches-py/test_batch")
test_data.keys()

dict_keys([b'batch_label', b'labels', b'data', b'filenames'])

In [7]:
test_images = test_data[b'data']
test_labels = test_data[b'labels']

In [8]:
images = images.reshape(-1, 3, 32, 32)
test_images = test_images.reshape(-1, 3, 32, 32)

In [9]:
images = np.transpose(images, (0, 2, 3, 1))
test_images = np.transpose(test_images, (0, 2, 3, 1))

extract local features

In [10]:
sift = cv2.SIFT_create()

In [11]:
def extract_local_features(images):
    descriptors_list = []
    for img in images:
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        gray = cv2.resize(gray, (64, 64), interpolation=cv2.INTER_CUBIC) # larger images needed for decent SIFT detection
        keypoints, descriptors = sift.detectAndCompute(gray, None)
        if descriptors is not None:
            descriptors_list.append(descriptors)
        else:
            descriptors_list.append(np.zeros((1, 128), dtype=np.float32))
    return descriptors_list

In [12]:
train_descriptors = extract_local_features(images)
test_descriptors = extract_local_features(test_images)

flatten the descriptors for clustering

In [13]:
all_descriptors = np.vstack([desc for desc in train_descriptors if desc is not None])

use K-Means to cluster feature descriptors into a fixed-length feature vector (Bag-of-Words approach)

In [14]:
num_clusters = 100  # number of visual words
kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init=10)
kmeans.fit(all_descriptors)

Exception in thread Thread-4 (_readerthread):
Traceback (most recent call last):
  File "C:\Users\70p6229\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\70p6229\computer_vision\examples\pytorch\.venv\lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\70p6229\AppData\Local\Programs\Python\Python310\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\70p6229\AppData\Local\Programs\Python\Python310\lib\subprocess.py", line 1515, in _readerthread
    buffer.append(fh.read())
  File "C:\Users\70p6229\AppData\Local\Programs\Python\Python310\lib\encodings\cp1252.py", line 23, in decode
    return codecs.charmap_decode(input,self.errors,decoding_table)[0]
UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 3: character maps to <undefined>
c:\Users\70p6229\computer_vision\examples\pytorch\.ve

KMeans(n_clusters=100, n_init=10, random_state=42)

convert feature descriptors into a histogram of visual words

In [15]:
def compute_bow_histograms(descriptors_list, kmeans_model):
    histograms = []
    for descriptors in descriptors_list:
        if descriptors is not None:
            words = kmeans_model.predict(descriptors)  # Assign descriptors to clusters
            hist, _ = np.histogram(words, bins=np.arange(num_clusters+1))  # Create histogram
        else:
            hist = np.zeros(num_clusters)  # If no descriptors, return zero vector
        histograms.append(hist)
    return np.array(histograms)

In [16]:
X_train = compute_bow_histograms(train_descriptors, kmeans)
X_test = compute_bow_histograms(test_descriptors, kmeans)

In [17]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [18]:
linear_est = LogisticRegression()
linear_est.fit(X_train, labels)

LogisticRegression()

In [19]:
yhat_train = linear_est.predict(X_train)
accuracy_score(labels, yhat_train)

0.33236

In [20]:
yhat_test = linear_est.predict(X_test)
accuracy_score(test_labels, yhat_test)

0.3314

In [21]:
knn_est = KNeighborsClassifier()
knn_est.fit(X_train, labels)

KNeighborsClassifier()

In [22]:
yhat_train = knn_est.predict(X_train)
accuracy_score(labels, yhat_train)

0.41636

In [23]:
yhat_test = knn_est.predict(X_test)
accuracy_score(test_labels, yhat_test)

0.2087

In [24]:
lgbm_est = HistGradientBoostingClassifier()
import os
os.environ['OMP_NUM_THREADS'] = '1'
lgbm_est.fit(X_train, labels)

HistGradientBoostingClassifier()

In [25]:
yhat_train = lgbm_est.predict(X_train)
accuracy_score(labels, yhat_train)

0.53

In [26]:
yhat_test = lgbm_est.predict(X_test)
accuracy_score(test_labels, yhat_test)

0.3299